In [2]:
import numpy as np
import d3rlpy
from d3rlpy.datasets import get_d4rl
import gym
from d3rlpy.metrics.scorer import evaluate_on_environment
import argparse
import os
from tqdm import tqdm


os.environ['D4RL_SUPPRESS_IMPORT_ERROR'] = '1'


/home/raunakk/miniconda3/envs/msc_project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
parser = argparse.ArgumentParser()
parser.add_argument("--task", type=str, help="task/game to be played")
parser.add_argument("--algo", type=str, help="algorithm to be used for training")
args = parser.parse_args()

task = args.task #['HalfCheetah-v4', 'Walker2d-v4', 'Ant-v4']
algo = args.algo
print(task)


usage: ipykernel_launcher.py [-h] [--task TASK] [--algo ALGO]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9008 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"9e8c588e-7ee3-41ea-b19d-c84d6b03e25b" --shell=9007 --transport="tcp" --iopub=9009 --f=/home/raunakk/.local/share/jupyter/runtime/kernel-v2-23007p03Mm7VoJj4E.json


SystemExit: 2

In [3]:
class MODEL():
    def __init__(self, task, algo, gpu=True):
        self.mean_results = []
        self.task = task 
        self.algo = algo
        self.f_params = {"use_gpu": gpu}
        self.engine = None

    def set_engine(self):
        if self.algo == "IQL":
            self.engine = d3rlpy.algos.IQL(**self.f_params)

        elif self.algo == "CQL":
            self.f_params["actor_learning_rate"] = 3e-5
            self.engine = d3rlpy.algos.CQL(**self.f_params)

        elif self.algo == "MOPO":
            self.engine = d3rlpy.algos.MOPO(**self.f_params)
            
        elif self.algo == "COMBO":
            self.engine = d3rlpy.algos.COMBO(**self.f_params)

    def train(self, n=100, n_steps=1000000, n_steps_per_epoch=10000, save_interval=100, save_metrics=False, verbose=False):
        dataset, env = get_d4rl(self.task)
        online_env = gym.make(self.task)
        for i in range(n):
            d3rlpy.seed(i)
            env.seed(i)
            online_env.seed(i)

            self.set_engine()

            self.engine.fit(dataset, n_steps=n_steps, n_steps_per_epoch=n_steps_per_epoch, save_interval=save_interval, save_metrics=save_metrics, verbose=verbose)
            self.engine.save_model("./saved_models/iql_{}_{}_{}.pt".format(algo, task, i))
            for i in range(100):
                scorer = evaluate_on_environment(online_env, n_trials=1)
                self.mean_results.append(scorer(self.engine))
        return self.mean_results

In [4]:
task = "halfcheetah-medium-v2"
algo = "CQL"
model = MODEL(task, algo)
mean_results = model.train(n=1, n_steps=1, n_steps_per_epoch=1)

<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
pybullet build time: Jun 13 2023 11:48:14
/home/raunakk/miniconda3/envs/msc_project/lib/python3.10/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
load datafile: 100%|██████████| 21/21 [00:03<00:00,  5.33it/s]
/home/raunakk/miniconda3/envs/msc_project/lib/python3.10/site-packages/gym/core.py:172: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed) instead.
  deprecation(


2023-06-24 18:20:14 [debug    ] RandomIterator is selected.
2023-06-24 18:20:14 [debug    ] Building models...
2023-06-24 18:20:57 [debug    ] Models have been built.


Epoch 1/1: 100%|██████████| 1/1 [00:33<00:00, 33.63s/it, temp_loss=8.4, temp=1, alpha_loss=-27.5, alpha=1, critic_loss=71.5, actor_loss=-2.36]


In [5]:
mean_results

[-1.3035401301234066,
 -1.0729566577045104,
 -3.9790731379671676,
 -1.5766215749340042,
 -0.8668231847213048,
 -1.703900797168258,
 -2.890828548706894,
 -0.3297628187275022,
 -1.9078329191056413,
 -2.077963004021463]

In [1]:
import d3rlpy
from d3rlpy.datasets import get_d4rl
from d3rlpy.dynamics import ProbabilisticEnsembleDynamics
from d3rlpy.metrics.scorer import dynamics_observation_prediction_error_scorer
from d3rlpy.metrics.scorer import dynamics_reward_prediction_error_scorer
from d3rlpy.metrics.scorer import dynamics_prediction_variance_scorer
from sklearn.model_selection import train_test_split
import gym
from tqdm.auto import trange

/home/raunakk/miniconda3/envs/msc_project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset, env = get_d4rl('hopper-medium-v2')

<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
No module named 'flow'
No module named 'carla'
pybullet build time: Jun 13 2023 11:48:14
/home/raunakk/miniconda3/envs/msc_project/lib/python3.10/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
load datafile: 100%|██████████| 21/21 [00:09<00:00,  2.24it/s]


In [9]:
dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics(learning_rate=1e-4, use_gpu=True)

train_episodes, test_episodes = train_test_split(dataset)
# same as algorithms
dynamics.fit(train_episodes,
             eval_episodes=test_episodes,
             n_epochs=1,
             save_interval=1,
             with_timestamp=False
)

2023-07-09 04:11:55 [debug    ] RoundIterator is selected.
2023-07-09 04:11:55 [info     ] Directory is created at d3rlpy_logs/ProbabilisticEnsembleDynamics
2023-07-09 04:11:55 [debug    ] Building models...
2023-07-09 04:23:21 [debug    ] Models have been built.
2023-07-09 04:23:26 [info     ] Parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics/params.json params={'action_scaler': None, 'batch_size': 100, 'discrete_action': False, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 1.0, 'generated_maxlen': 100000, 'learning_rate': 0.0001, 'n_ensembles': 5, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0.0001, 'amsgrad': False}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': None, 'use_gpu': 0, 'variance_type': 'max', 'algorithm': 'ProbabilisticEnsembleDynamics', 'observation_shape': (11,), 'action_size': 3}


Epoch 1/1: 100%|██████████| 7464/7464 [07:37<00:00, 16.32it/s, loss=-71.5] 


2023-07-09 04:31:04 [info     ] ProbabilisticEnsembleDynamics: epoch=1 step=7464 epoch=1 metrics={'time_sample_batch': 0.01169888025183642, 'time_algorithm_update': 0.0480932249964114, 'loss': -71.47799279192904, 'time_step': 0.06030370510659417} step=7464
2023-07-09 04:31:05 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics/model_7464.pt


[(1,
  {'time_sample_batch': 0.01169888025183642,
   'time_algorithm_update': 0.0480932249964114,
   'loss': -71.47799279192904,
   'time_step': 0.06030370510659417})]

In [11]:
# load trained dynamics model
dynamics = ProbabilisticEnsembleDynamics.from_json('./d3rlpy_logs/ProbabilisticEnsembleDynamics/params.json')
dynamics.load_model('./d3rlpy_logs/ProbabilisticEnsembleDynamics/model_7464.pt')

# give mopo as generator argument.
mopo = d3rlpy.algos.MOPO(dynamics=dynamics)

2023-07-09 04:34:04 [warning  ] Unused arguments are passed.   algorithm=ProbabilisticEnsembleDynamics gamma=1.0 generated_maxlen=100000 n_steps=1 real_ratio=1.0
